Install dependencies

In [ ]:
!pip install web3

# Below installs are specific for the latest version of web3 to work with google colab
!pip install protobuf==3.17.3
!pip install eth-tester

In [ ]:
# used to print all outputs from a cell
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"
# default value is: "last_expr"

Connect to Infura

In [ ]:
from web3 import Web3

infura_url = ""
w3 = Web3(Web3.HTTPProvider(infura_url))
  
print(w3.isConnected())

True


Connect to smart contract

In [ ]:
abi = [{"inputs":[{"internalType":"bytes32","name":"_password","type":"bytes32"}],"name":"addUser","outputs":[],"stateMutability":"nonpayable","type":"function"},{"inputs":[{"internalType":"bytes32","name":"_password","type":"bytes32"}],"stateMutability":"nonpayable","type":"constructor"},{"inputs":[],"name":"count","outputs":[{"internalType":"uint256","name":"","type":"uint256"}],"stateMutability":"view","type":"function"},{"inputs":[{"internalType":"uint256","name":"","type":"uint256"}],"name":"data","outputs":[{"internalType":"bytes32","name":"","type":"bytes32"}],"stateMutability":"view","type":"function"},{"inputs":[{"internalType":"uint256","name":"slot","type":"uint256"},{"internalType":"uint256","name":"index","type":"uint256"},{"internalType":"uint256","name":"elementSize","type":"uint256"}],"name":"getArrayLocation","outputs":[{"internalType":"uint256","name":"","type":"uint256"}],"stateMutability":"pure","type":"function"},{"inputs":[{"internalType":"uint256","name":"slot","type":"uint256"},{"internalType":"uint256","name":"key","type":"uint256"}],"name":"getMapLocation","outputs":[{"internalType":"uint256","name":"","type":"uint256"}],"stateMutability":"pure","type":"function"},{"inputs":[],"name":"isTrue","outputs":[{"internalType":"bool","name":"","type":"bool"}],"stateMutability":"view","type":"function"},{"inputs":[],"name":"owner","outputs":[{"internalType":"address","name":"","type":"address"}],"stateMutability":"view","type":"function"},{"inputs":[],"name":"someConst","outputs":[{"internalType":"uint256","name":"","type":"uint256"}],"stateMutability":"view","type":"function"},{"inputs":[],"name":"u16","outputs":[{"internalType":"uint16","name":"","type":"uint16"}],"stateMutability":"view","type":"function"}]
addr = "0x18a975685Bf6e76604d221ce43A347946848C164"

contract = w3.eth.contract(address=addr, abi=abi)
contract

Slot 0

In [ ]:
w3.eth.getStorageAt(addr, 0)
display(int(0x7b))

HexBytes('0x000000000000000000000000000000000000000000000000000000000000007b')

123

slot 1 - u16, isTrue, owner

In [ ]:
w3.eth.getStorageAt(addr, 1)
# u16 = 1f
# boolean value = 01
# owner addr = your-wallet-address
display(int(0x1f))

HexBytes('0x000000000000000000001f015da4821fdaf8395d42805bc67f44a01e90c9e5b6')

31

slot 2 - password

In [ ]:
# Get the HexBytes in slot 2nd
print('Content in slot 2:')
display(w3.eth.getStorageAt(addr, 2))

### Option 1
print('\nOption 1')
def hex_to_string(hex):
    if hex[:2] == '0x':
        hex = hex[2:]
    string_value = bytes.fromhex(str(hex)).decode('utf-8')
    return string_value

# get hex sring from hexbytes object
print(' Isolate the hex value from the HexBytes object')
hexSlot2 = w3.eth.getStorageAt(addr, 2).hex()
display(hexSlot2)

# convert to human readable value
print('\n Get User1 password usign custom function')
pwdSlot2 = hex_to_string(hexSlot2)
display(pwdSlot2)

### Option 2
print('\nOption 2: using built-in python function')
# Use python built-in function to convert to ascii; in javascript: web3.utils.toAscii("...")
display(str(w3.eth.getStorageAt(addr, 2)))

Content in slot 2:


HexBytes('0x7468697369736173757065727374726f6e6770617373776f7264000000000000')


Option 1
 Isolate the hex value from the HexBytes object


'0x7468697369736173757065727374726f6e6770617373776f7264000000000000'


 Get User1 password usign custom function


'thisisasuperstrongpassword\x00\x00\x00\x00\x00\x00'


Option 2: using built-in python function


"b'thisisasuperstrongpassword\\x00\\x00\\x00\\x00\\x00\\x00'"

slot 6 - array length 

In [ ]:
# how many users have ben stored ?
w3.eth.getStorageAt(addr, 6)

HexBytes('0x0000000000000000000000000000000000000000000000000000000000000002')

User 1

In [ ]:
# Get the location of the 1st array element
print('Where is the array\'s 1st item stored ?')

# Option 1:
print('\nOption 1: using custom solidity function')
loc = contract.functions.getArrayLocation(6, 0, 2).call()
print(f' User 1 data location: {loc}')

hexloc = w3.toHex(loc)
print(f'\nUser 1 data location in hexadecimals:', hexloc)

# Option 2:
print('\nOption 2: soliditySha3')
hash_user1_location = w3.soliditySha3(['uint256'], [6])
display(hash_user1_location)

# Get the ID of the first user
print('\nUse hash_user1_location to get the ID of the first User')
print(' ID of the first User:')
display(w3.eth.getStorageAt(addr, hash_user1_location))

# Get hash of User 1 password location: increment hash of ID by 1
print('\nGet hash of User 1 password location: increment hash of ID by 1')
hash_user1_PWD_location = "0xf652222313e28459528d920b65115c16c04f3efc82aaedc97be59f3f377c0d40"
display(hash_user1_PWD_location)
print('\n ..3f becomes ..40')

Where is the array's 1st item stored ?

Option 1: using custom solidity function
 User 1 data location: 111414077815863400510004064629973595961579173665589224203503662149373724986687

User 1 data location in hexadecimals: 0xf652222313e28459528d920b65115c16c04f3efc82aaedc97be59f3f377c0d3f

Option 2: soliditySha3


HexBytes('0xf652222313e28459528d920b65115c16c04f3efc82aaedc97be59f3f377c0d3f')


Use hash_user1_location to get the ID of the first User
 ID of the first User:


HexBytes('0x0000000000000000000000000000000000000000000000000000000000000000')


Get hash of User 1 password location: increment hash of ID by 1


'0xf652222313e28459528d920b65115c16c04f3efc82aaedc97be59f3f377c0d40'


 ..3f becomes ..40


In [ ]:
# convert new hash to human readable value
print('\n Get User1 password')
hexstr1 = "0xf652222313e28459528d920b65115c16c04f3efc82aaedc97be59f3f377c0d40"
str(w3.eth.getStorageAt(addr, hexstr1))


 Get User1 password


"b'thisisuser1password\\x00\\x00\\x00\\x00\\x00\\x00\\x00\\x00\\x00\\x00\\x00\\x00\\x00'"

User 2

In [ ]:
# increment hash_user1_PWD_location by 1 to get User 2 ID location
print('\nIncrement hash_user1_PWD_location by 1 to get User 2 ID location')
hash_user2_ID_loc = "0xf652222313e28459528d920b65115c16c04f3efc82aaedc97be59f3f377c0d41"
display(hash_user2_ID_loc)

# Get the ID of the 2nd user
print('\nGet ID User 2 from new hash:')
display(w3.eth.getStorageAt(addr, hash_user2_ID_loc))

# increment hash of User 2 ID location by 1 to get User 2 password location
print('\nIncrement hash of User 2 ID location by 1 to get User 2 password location')
hash_user2_PWD_location = "0xf652222313e28459528d920b65115c16c04f3efc82aaedc97be59f3f377c0d42"
display(hash_user2_PWD_location)

# convert to human readable value
print('\nGet User2 password')
str(w3.eth.getStorageAt(addr, hash_user2_PWD_location))


Increment hash_user1_PWD_location by 1 to get User 2 ID location


'0xf652222313e28459528d920b65115c16c04f3efc82aaedc97be59f3f377c0d41'


Get ID User 2 from new hash:


HexBytes('0x0000000000000000000000000000000000000000000000000000000000000001')


Increment hash of User 2 ID location by 1 to get User 2 password location


'0xf652222313e28459528d920b65115c16c04f3efc82aaedc97be59f3f377c0d42'


Get User2 password


"b'user2megastrongpassword\\x00\\x00\\x00\\x00\\x00\\x00\\x00\\x00\\x00'"

slot 7 - empty

In [ ]:
w3.eth.getStorageAt(addr, 7)

HexBytes('0x0000000000000000000000000000000000000000000000000000000000000000')

Mapping

In [ ]:
# get hex of the map location of User 2

print('Slot where User 2 entry of the mapping is stored:')
# Option 1
print('\nOption 1: with custom function')
mapHexOpt1 = contract.functions.getMapLocation(7, 1).call()
display(mapHexOpt1)

mapHexLocation = w3.toHex(mapHexOpt1)
display(mapHexLocation)

# Option 2
print('\nOption 2: with soliditySha3')
mapHexOpt2 = w3.soliditySha3(['uint256', 'uint256'], [1, 7])
display(mapHexOpt2)

Slot where User 2 entry of the mapping is stored:

Option 1: with custom function


81222191986226809103279119994707868322855741819905904417953092666699096963112

'0xb39221ace053465ec3453ce2b36430bd138b997ecea25c1043da0c366812b828'


Option 2: with soliditySha3


HexBytes('0xb39221ace053465ec3453ce2b36430bd138b997ecea25c1043da0c366812b828')

In [ ]:
# Get the ID of the 1st user in the mapping (ID=1)
print('Get the hash of the value stored at that location')
print(" It\'s the ID of user 2 in mapping")
User2ID = w3.eth.getStorageAt(addr, mapHexOpt2)
display(User2ID)

# increment data location by 1 to get the password location
print('\nIncrement data location of User by 1')
User2PwdLoc = '0xb39221ace053465ec3453ce2b36430bd138b997ecea25c1043da0c366812b829'
display(User2PwdLoc)

Get the hash of the value stored at that location
 It's the ID of user 2 in mapping


HexBytes('0x0000000000000000000000000000000000000000000000000000000000000001')


Increment data location of User by 1


'0xb39221ace053465ec3453ce2b36430bd138b997ecea25c1043da0c366812b829'

In [ ]:
# convert to human readable value
print('\nUser2 pwd is:')
str(w3.eth.getStorageAt(addr, User2PwdLoc))


User2 pwd is:


"b'user2megastrongpassword\\x00\\x00\\x00\\x00\\x00\\x00\\x00\\x00\\x00'"